In [1]:
import pygame
import random
import math
import numpy as np
import time

pygame 2.1.2 (SDL 2.0.18, Python 3.8.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [205]:
SAFETY_DISTANCE = 20
KILL_DISTANCE = 30

In [223]:
class Car:
    def __init__(self, next_stop = None, current_traj = None): #pos input is a tuple
        self.pos = None
        self.direction = None
        self.distance = None
        self.next = None
        self.prev = None
        self.next_stop = next_stop
        self.current_traj = current_traj
        
        self.speed = 2
        self.max_speed = 4
        self.acceleration = 0.02
        self.deceleration = 0
        self.lookahead = self.speed * 20

        
        self.color = (255, 255, 255)
        
    def determine_distance(self):
        if self.next_stop.red == True:
            return min(math.dist(self.pos, self.next.pos), 
                       math.dist(self.pos, self.next_stop.pos))
        else:
            return math.dist(self.pos, self.next.pos)
    
    def step(self):
        new_distance = self.determine_distance()
        
        #kill
        if self.next_stop.threshold == True and new_distance < KILL_DISTANCE:
            try:
                self.prev.next = self.next_stop
            except:
                pass
            return
        
        #new stoplight
        if np.sum(np.sign(self.next_stop.pos - self.pos) + np.sign(self.direction)) == 0:
            self.next_stop = self.next_stop.next
            self.current_traj = self.current_traj.next

        if new_distance < self.lookahead + SAFETY_DISTANCE:
            self.speed = max(self.speed/2, 0)
        else:
            self.speed = min(self.speed+self.acceleration, self.max_speed)
            
        
        self.direction = self.current_traj.direction
        self.distance = new_distance
        self.lookahead = self.speed * 20
        self.pos = self.pos + (self.speed*self.direction)

In [217]:
class Stoplight:
    def __init__(self, pos, red = True, threshold = False): #pos input is a tuple
        self.pos = np.array(pos)
        self.next = None
        self.red = red
        self.threshold = threshold
        self.get_color()
        
    def step(self):
        self.red = not self.red
        self.get_color()
        
    def get_color(self):
        if self.red == True:
            self.color = (255, 0, 0)
        else:
            self.color = (0, 255, 0)
            
    def get_last(self):
        a = self
        while a.next != None:
            a = a.next
        return a

In [208]:
class Trajectory:
    def __init__(self, start, end): #pos input is a tuple
        self.start = np.array(start)
        self.end = np.array(end)
        self.next = None
        self.direction = self.get_direction()
        
    def get_direction(self):
        direction_not_norm = self.end - self.start
        return direction_not_norm/np.sqrt(np.sum((direction_not_norm)**2))

In [209]:
#at first we need to create the stoplight linked list
class Stop_list:
    
    def __init__(self, head = None):
        self.head = head
        
    def __repr__(self):
        lst = []
        self._aid_repr(self.head, lst)
        return ' '.join(lst)
    
    def _aid_repr(self, node, lst):
        if node:
            lst.append(str(node.pos))
            self._aid_repr(node.next, lst)
    
    def append(self, other):
        if self.head == None:
            self.head = other
            return
        a = self.head
        while a.next != None:
            a = a.next
        a.next = other

In [210]:
#creating the trajectory linked list
class Traj_list:
    
    def __init__(self, head = None):
        self.head = head
        
    def __repr__(self):
        lst = []
        self._aid_repr(self.head, lst)
        return ' '.join(lst)
    
    def _aid_repr(self, node, lst):
        if node:
            lst.append(str(node.pos))
            self._aid_repr(node.next, lst)
        
    def append(self, other):
        if self.head == None:
            self.head = other
            return
        a = self.head
        while a.next != None:
            a = a.next
        a.next = other

In [211]:
#lastly the car linked list
class Car_list:
    
    def __init__(self, head = None):
        self.head = head
        
    def __repr__(self):
        lst = []
        self._aid_repr(self.head, lst)
        return ' '.join(lst)
    
    def _aid_repr(self, node, lst):
        if node:
            lst.append(str(node.pos))
            self._aid_repr(node.next, lst)
        
    def front_append(self, other):
        #appending the car
        if self.head == None:
            self.head = other
            self.head.next = other.next_stop.get_last()
            #initializing the other details of the car
            other.pos = other.current_traj.start
            other.direction = other.current_traj.direction
            other.distance = other.determine_distance()
            return
        
        self.head.prev = other
        a = self.head
        other.next = a
        self.head = other
        #initializing the other details of the car
        other.pos = other.current_traj.start
        other.direction = other.current_traj.direction
        other.distance = other.determine_distance()

In [ ]:
time.sleep(3)
pygame.init()
screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Traffic Simulation")
img = pygame.image.load("intersection.png").convert()

stl = []
trl = []
cal = []

sl1 = Stop_list()
tl1 = Traj_list()
cl1 = Car_list()
sl1.append(Stoplight((400, 300)))
sl1.append(Stoplight((400, 0), red = False, threshold = True)) #fake stoplight
tl1.append(Trajectory((400, 600), (400, 300)))
tl1.append(Trajectory((400, 300), (400, 0)))

stl.append(sl1)
trl.append(tl1)
cal.append(cl1)

sl2 = Stop_list()
tl2 = Traj_list()
cl2 = Car_list()
sl2.append(Stoplight((380, 300)))
sl2.append(Stoplight((0, 300), red = False, threshold = True)) #fake stoplight
tl2.append(Trajectory((380, 600), (380, 300)))
tl2.append(Trajectory((380, 300), (0, 300)))

stl.append(sl2)
trl.append(tl2)
cal.append(cl2)

i = 0
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
    screen.fill((100, 100, 100))
    
    for sl, tl, cl in zip(stl, trl, cal):
        #stoplights
        if i%200 == 0:
            sl.head.step()
        stoplight = sl.head
        while stoplight != None:
            pygame.draw.circle(screen, stoplight.color, stoplight.pos, 8)
            stoplight = stoplight.next
    
        #cars
            if min(np.random.poisson(0.01), 1) == 1: #rate at which to spawn cars
                cl.front_append(Car(sl.head, tl.head))
            if cl.head != None:
                car = cl.head
                while car.next != None:
                    car.step()
                    pygame.draw.circle(screen, car.color, car.pos, 4)
                    car = car.next
            
    pygame.display.update()
    i += 1

In [147]:
pygame.quit()